# Demo - MAB test with Thompson sampling

This demo uses the dataset from [Kaggle/ab-testing-dataset](https://www.kaggle.com/datasets/amirmotefaker/ab-testing-dataset/) to simulate the whole situation again and compare the result of the MAB and AB tests by limiting the daily sample size to 10k impressions. For the AB test, both variants always split the sample size equally (5k). Whenever the number of impressions in the data is less than the desired sample size, 5k in AB test or the calculated size in MAB, we assume that we can only collect the number of samples in the given data, with the concern that the CPI may increase as the number of impressions increases.

### Objective:
Optimise the number of purchases at the lowest cost

### Assumptions:
- The cost, Spend[USD], is calculated by the number of impressions.
- CPI (cost per impression) is always constant at the same time when fewer impressions have been obtained.

In [1]:
# Load data
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

control = pd.read_csv('sample_data/ABTestDataSet/control_group.csv', sep=';')
test = pd.read_csv('sample_data/ABTestDataSet/test_group.csv', sep=';')

df = pd.concat([control, test], axis=0) 
df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x, "%d.%m.%Y"))
pd.concat([df.head(3), df.tail(3)], axis=0)

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
0,Control Campaign,2019-08-01,2280,82702.0,56930.0,7016.0,2290.0,2159.0,1819.0,618.0
1,Control Campaign,2019-08-02,1757,121040.0,102513.0,8110.0,2033.0,1841.0,1219.0,511.0
2,Control Campaign,2019-08-03,2343,131711.0,110862.0,6508.0,1737.0,1549.0,1134.0,372.0
27,Test Campaign,2019-08-28,2247,54627.0,41267.0,8144.0,2432.0,1281.0,1009.0,721.0
28,Test Campaign,2019-08-29,2805,67444.0,43219.0,7651.0,1920.0,1240.0,1168.0,677.0
29,Test Campaign,2019-08-30,1977,120203.0,89380.0,4399.0,2978.0,1625.0,1034.0,572.0


In [2]:
df[df.Date == '2019-08-05']

,Campaign Name,Date,Spend [USD],# of Impressions,Reach,# of Website Clicks,# of Searches,# of View Content,# of Add to Cart,# of Purchase
4,Control Campaign,2019-08-05,1835,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Test Campaign,2019-08-05,2297,114295.0,95138.0,5863.0,2106.0,858.0,956.0,768.0


There are missing value on 2019-08-05, therefore we will drop this date in the following experiment.

In [3]:
# Drop rows with Date == '2019-08-05'
df = df.loc[df['Date'] != '2019-08-05']

In [4]:
# Add different KPIs
df['purchase_rate']=df['# of Purchase']/df['# of Impressions']
df['cpi'] = df['Spend [USD]']/df['# of Impressions']
df['cost per purchase'] = df['Spend [USD]']/df['# of Purchase']

In [23]:
# Seting test parameters
reward_type = 'Cost per Purchase'
reward_col = 'cost per purchase'
batch_col='Date'
sample_size = 100000

In [18]:
df.groupby('Campaign Name')['Spend [USD]'].sum()/df.groupby('Campaign Name')['# of Purchase'].sum()

Campaign Name
Control Campaign    4.407229
Test Campaign       5.016814
dtype: float64

In [19]:
variant_id = df['Campaign Name'].unique()

# preset parameters for testing. Assume that the prior is strong and the same for all variants
prior_hist = {variant: {'alpha': 100, 'beta': 100} for variant in variant_id}
experiment_prior = {}
experiment_prior= prior_hist
experiment_prior

{'Control Campaign': {'alpha': 100, 'beta': 100},
 'Test Campaign': {'alpha': 100, 'beta': 100}}

In [22]:
df_pivoted = df.pivot(index='Date', columns='Campaign Name', values=reward_col)
df_pivoted['Test>Control'] = df_pivoted['Test Campaign']>df_pivoted['Control Campaign']
df_pivoted['Test>Control'].mean()

0.5172413793103449

In [24]:
# Run the simulation for 1 time
from mab_utils import scenario_gen
weight_group, data_size_group, params_list = scenario_gen(
    df, reward_type, reward_col, experiment_prior, batch_col, 
    iteration = 1)

Waring: The sample size, 89550 is not equal to the target sample size, 100000.
Waring: The sample size, 90468 is not equal to the target sample size, 100000.
Waring: The sample size, 99999 is not equal to the target sample size, 100000.
Waring: The sample size, 99999 is not equal to the target sample size, 100000.
Waring: The sample size, 99999 is not equal to the target sample size, 100000.
Waring: The sample size, 77721 is not equal to the target sample size, 100000.
Waring: The sample size, 95370 is not equal to the target sample size, 100000.
Waring: The sample size, 91751 is not equal to the target sample size, 100000.
Waring: The sample size, 94149 is not equal to the target sample size, 100000.
Waring: The sample size, 99999 is not equal to the target sample size, 100000.


In [48]:
original_performance = df.pivot(index='Date', columns='Campaign Name', values=[reward_col, '# of Impressions'])
original_performance.index = original_performance.index.astype('str') 
performance_comp = pd.concat([
    pd.DataFrame(data_size_group[0]).transpose(),
    original_performance], axis=1)

import numpy as np
performance_comp['MAB Performance'] = performance_comp['Test Campaign']*performance_comp[(reward_col, 'Test Campaign')]+performance_comp['Control Campaign']*performance_comp[(reward_col, 'Control Campaign')]
performance_comp['Original Performance'] = np.minimum(performance_comp[('# of Impressions', 'Test Campaign')], sample_size / 2)*performance_comp[(reward_col, 'Test Campaign')]+np.minimum(performance_comp[('# of Impressions', 'Control Campaign')], sample_size / 2)*performance_comp[(reward_col, 'Control Campaign')]
performance_comp['MAB_size'] = performance_comp['Test Campaign']+performance_comp['Control Campaign']
performance_comp['Original_size'] = np.minimum(performance_comp[('# of Impressions', 'Test Campaign')], sample_size / 2)+np.minimum(performance_comp[('# of Impressions', 'Control Campaign')], sample_size / 2)
avg_diff = performance_comp['MAB Performance'].sum()/performance_comp['MAB_size'].sum() - performance_comp['Original Performance'].sum()/performance_comp['Original_size'].sum()

In [55]:
print(f"Performance of MAB comparing to AB test in {reward_type}: €{avg_diff}, with {performance_comp['MAB_size'].sum()-performance_comp['Original_size'].sum()} additonal impressions.")

Performance of MAB comparing to AB test in Cost per Purchase: €-0.01181271174002596, with 44198.0 additonal impressions.


# Summary:

The MAB test is almost stablised after 4 days and it has the same result as the AB test where the control campaign has a lower cost per purchase. However, the MAB test reduce the cost per purchase by €0.012 while having 44k more impression.

In [53]:
# Run the simulation for 100 times
from mab_utils import scenario_gen
weight_group, data_size_group, params_list = scenario_gen(
    df, reward_type, reward_col, experiment_prior, batch_col, 
    iteration = 100)

Waring: The sample size, 89550 is not equal to the target sample size, 100000.
Waring: The sample size, 91568 is not equal to the target sample size, 100000.
Waring: The sample size, 99999 is not equal to the target sample size, 100000.
Waring: The sample size, 99999 is not equal to the target sample size, 100000.
Waring: The sample size, 99999 is not equal to the target sample size, 100000.
Waring: The sample size, 99999 is not equal to the target sample size, 100000.
Waring: The sample size, 79220 is not equal to the target sample size, 100000.
Waring: The sample size, 95270 is not equal to the target sample size, 100000.
Waring: The sample size, 90452 is not equal to the target sample size, 100000.
Waring: The sample size, 93949 is not equal to the target sample size, 100000.
Waring: The sample size, 89550 is not equal to the target sample size, 100000.
Waring: The sample size, 92469 is not equal to the target sample size, 100000.
Waring: The sample size, 99999 is not equal to the t